In [11]:
### Book to get game time weather conditions for each game in the dataset

from bs4 import BeautifulSoup as BS
from selenium import webdriver
from functools import reduce
import pandas as pd
import time
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from selenium import webdriver
# from webdriver_manager.chrome import ChromeDriverManager

# driver = webdriver.Chrome(ChromeDriverManager().install())



In [12]:
## The jumping off point for this methos is a post and code from 
## https://bojanstavrikj.github.io/content/page1/wunderground_scraper_daily

## This seems to work using the 



In [13]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options

## Base Code
def render_page(url,type):
    # Setup Chrome options
    chrome_options = Options()
    
    # Instantiate the Webdriver: Mention the executable path of the webdriver you have downloaded
    # If it's in the PATH, you don't need to provide a path string
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)
    
    driver.get(url)
    time.sleep(3)
    return driver.page_source
            

In [14]:
def hourly_scraper(page,dates,type):
    output = pd.DataFrame()

    for d in dates:
        url = str(str(page) + str(d))

        r = render_page(url,type)

        soup = BS(r, "html.parser",)
        container = soup.find('lib-city-history-observation')
        check = container.find('tbody')

        data = []
        data_hour = []
        for i in check.find_all('span', class_='ng-star-inserted'):
            trial = i.get_text()
            data_hour.append(trial)

        for i in check.find_all('span', class_='wu-value wu-value-to'):
            trial = i.get_text()
            data.append(trial)


        numbers = pd.DataFrame([data[i:i+7] for i in range(0, len(data), 7)],columns=["Temperature","Dew Point","Humidity","Wind Speed","Wind Gust","Pressure","Precipitation"])
        hour = pd.DataFrame(data_hour[0::17],columns=["Time"])
        wind = pd.DataFrame(data_hour[7::17],columns=["Wind"])
        condition = pd.DataFrame(data_hour[16::17],columns=["Condition"])

        dfs = [hour,numbers,wind,condition]

        df_final = reduce(lambda left, right: pd.merge(left, right, left_index=True, right_index=True), dfs)
        df_final['Date'] = str(d)

        output = output.append(df_final)
        print(str(str(d) + ' finished!'))

    return output


page = "https://www.wunderground.com/history/daily/KMIEASTL78/date/"
dates = ["2020-06-27"]

hourly = hourly_scraper(page,dates,"F")

2020-06-27 finished!


C:\Users\Justin\AppData\Local\Temp\ipykernel_8088\530071273.py:34: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  output = output.append(df_final)


In [15]:
print(hourly)

        Time Temperature Dew Point Humidity Wind Speed Wind Gust Pressure  \
0   12:33 AM          67        66       97         12         0    28.87   
1   12:39 AM          67        65       93          9         0    28.87   
2   12:53 AM          67        65       93          8         0    28.87   
3    1:06 AM          67        65       93         12         0    28.87   
4    1:53 AM          67        65       93          9         0    28.88   
5    2:09 AM          67        65       93          8         0    28.88   
6    2:53 AM          68        66       93          7         0    28.87   
7    3:22 AM          68        66       93          7         0    28.86   
8    3:39 AM          68        66       93          9         0    28.86   
9    3:53 AM          69        67       93          9         0    28.87   
10   4:11 AM          69        67       93         10         0    28.87   
11   4:53 AM          69        67       93          9         0    28.88   